**<mark>Download from the web</mark>**

In [13]:
Nbr_Files_to_Download = 70

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 15, Finished, Available)

In [14]:
import pandas as pd
import re ,shutil,glob
from urllib.request import urlopen
import os
import requests
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa
def load(url,Path,x):
    #run only once to create an empty log file
    if not os.path.exists(Path):
      os.makedirs(Path, exist_ok=True) 
      os.makedirs(Path+"_log", exist_ok=True)
      log_tb = pa.Table.from_pylist( ['x'], schema=pa.schema({ "file" : pa.string()}))
      pq.write_table(log_tb,Path+"_log/log.parquet")  
    # Regex don't ask I just copy it
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist_sorted=sorted(filelist_unique, reverse=True)
    filelist = filelist_sorted[:x]
    ### Read from existing log
    df = ds.dataset(Path + "_log/log.parquet").to_table().to_pandas()     
    file_loaded= df['file'].unique()
    current = file_loaded.tolist()
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload)) 
    print(str(len(files_to_upload)) + ' New File Loaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"{Path}/{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
      
    #print(log)
    L=[os.path.basename(x) for x in glob.glob(Path+'*.zip')]
    log = pd.DataFrame({'file':L})
    log_tb=pa.Table.from_pandas(log,preserve_index=False)
    #print(log_tb)
    log_schema = pa.schema([pa.field('file', pa.string())])
    log_tb=log_tb.cast(target_schema=log_schema)
    pq.write_table(log_tb,Path+"_log/log.parquet")
    return "done"

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 16, Finished, Available)

In [15]:
load("https://data.wa.aemo.com.au/public/market-data/wemde/facilityScada/previous/","/lakehouse/default/Files/0_Source/WA/facilityScada/",Nbr_Files_to_Download)

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 17, Finished, Available)

0 New File Loaded


'done'

In [16]:
load("https://data.wa.aemo.com.au/public/market-data/wemde/facilityScada/current/","/lakehouse/default/Files/0_Source/WA/facilityScada/",Nbr_Files_to_Download)

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 18, Finished, Available)

0 New File Loaded


'done'

In [17]:
load("https://nemweb.com.au/Reports/Current/Daily_Reports/","/lakehouse/default/Files/0_Source/ARCHIVE/Daily_Reports/",Nbr_Files_to_Download)

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 19, Finished, Available)

0 New File Loaded


'done'

**<mark>Unzip</mark>**

In [18]:
import pandas as pd
from shutil import unpack_archive
import os
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa
def unzip(Source, Destination):
    #run only once to create an empty log file
    if not os.path.exists(Destination):
      os.makedirs(Destination, exist_ok=True) 
      os.makedirs(Destination+"_log", exist_ok=True)
      log_tb = pa.Table.from_pylist( ['x'], schema=pa.schema({ "file" : pa.string()}))
      pq.write_table(log_tb,Destination+"_log/log.parquet")  
    # check zip files
    try:
     df = ds.dataset(Source + "_log/log.parquet").to_table().to_pandas()     
     filelist_unique= df['file'].unique()
     filelist=filelist_unique.tolist()
    except: 
     filelist=[os.path.basename(x) for x in glob.glob(Source+'*.zip')]
    ### checl the unzipped files already
    df = ds.dataset(Destination + "_log/log.parquet").to_table().to_pandas()     
    file_loaded= df['file'].unique()
    current = file_loaded.tolist()
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload)) 
    #unzip only the delta
    print(str(len(files_to_upload)) + ' New File Loaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
          try:
            unpack_archive(str(Source+x), str(Destination), 'zip')
          except:
            pass
      existing_file = pd.DataFrame( file_loaded)
      new_file = pd.DataFrame(  files_to_upload)
      log = pd.concat ([new_file,existing_file], ignore_index=True)
      #print(log)
      log.rename(columns={0: 'file'}, inplace=True)
      log_tb=pa.Table.from_pandas(log,preserve_index=False)
      #print(log_tb)
      log_schema = pa.schema([pa.field('file', pa.string())])
      log_tb=log_tb.cast(target_schema=log_schema)
      pq.write_table(log_tb,Destination+"_log/log.parquet")
      return "done"
    else:
     return "nothing to see here"

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 20, Finished, Available)

In [19]:
unzip("/lakehouse/default/Files/0_Source/ARCHIVE/Daily_Reports/","/lakehouse/default/Files/1_Transform/0/ARCHIVE/Daily_Reports/")

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 21, Finished, Available)

0 New File Loaded


'nothing to see here'

In [20]:
unzip("/lakehouse/default/Files/0_Source/WA/facilityScada/","/lakehouse/default/Files/1_Transform/0/WA/facilityScada/")

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 22, Finished, Available)

0 New File Loaded


'nothing to see here'

In [21]:
unzip("/lakehouse/default/Files/archive_daily_reports/","/lakehouse/default/Files/1_Transform/0/ARCHIVE/Daily_Reports/")

StatementMeta(, fd5879fc-96fb-4862-8975-2b4b5119d456, 23, Finished, Available)

0 New File Loaded


'nothing to see here'